In [1]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../datasets/BankChurners_change.csv', encoding='CP949')
data = data.iloc[:,2:23]
np.random.seed(42)
random.seed(42)

In [3]:
object_columns = data.select_dtypes('object').columns
for i in object_columns:
    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']

category : [0 1]
classes : ['F' 'M']

category : [0 1 2 3 4 5 6]
classes : ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']

category : [0 1 2 3]
classes : ['Divorced' 'Married' 'Single' 'Unknown']

category : [0 1 2 3 4 5]
classes : ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K'
 'Unknown']

category : [0 1 2 3]
classes : ['Blue' 'Gold' 'Platinum' 'Silver']

category : [0 1 2 3 4]
classes : ['20대' '30대' '40대' '50대' '60대 이상']



## Under Sampling

In [4]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i == OneSidedSelection:
        sampling = i(random_state=42)
    else:
        sampling = i()
    
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    lr = LogisticRegression(random_state=42)
    lr.fit(x_train_ss, y_train)
    pred = lr.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Logistic Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

Logistic Model's score by Under sampling


,f1_score,recall,precision,accuracy
OneSidedSelection,0.942324,0.966451,0.919373,0.900790
TomekLinks,0.942324,0.966451,0.919373,0.900790
NeighbourhoodCleaningRule,0.941725,0.951148,0.932487,0.901283
EditedNearestNeighbours,0.937793,0.940553,0.935050,0.895360
AllKNN,0.936597,0.934667,0.938534,0.893880
RepeatedEditedNearestNeighbours,0.934795,0.928193,0.941493,0.891412
NearMiss,0.858991,0.781636,0.953338,0.784798


## Over Sampling

In [5]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy, random_state=42)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        lr = LogisticRegression(random_state=42)
        lr.fit(x_train_ss, y_train)
        pred = lr.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("Logistic Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Logistic Model's score by Over sampling


,f1_score,recall,precision,accuracy
ADASYN(0.3),0.942308,0.951736,0.933064,0.902270
SMOTE(0.3),0.942067,0.952325,0.932028,0.901777
BorderlineSMOTE(0.3),0.940556,0.945262,0.935897,0.899803
SMOTE(0.4),0.934757,0.927604,0.942020,0.891412
BorderlineSMOTE(0.4),0.934250,0.919953,0.948998,0.891412
ADASYN(0.4),0.932858,0.924073,0.941812,0.888450
SMOTE(0.5),0.931737,0.915833,0.948202,0.887463
SMOTE(0.6),0.930275,0.907004,0.954771,0.885982
BorderlineSMOTE(0.5),0.929994,0.907004,0.954180,0.885489
ADASYN(0.5),0.928939,0.904061,0.955224,0.884008


## Combine Sampling

In [6]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i(random_state=42)
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    lr = LogisticRegression(random_state=42)
    lr.fit(x_train_ss, y_train)
    pred = lr.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Logistic Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

Logistic Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTETomek,0.909994,0.862861,0.962574,0.856861
SMOTEENN,0.882617,0.809888,0.969697,0.819348


## Pipeline을 통한 Combine Sampling

In [7]:
strategy_ = [0.3, 0.4, 0.5]
columns = ['f1_score', 'recall', 'precision', 'accuracy']

df = pd.DataFrame(columns=columns)

under_sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]
over_sampling = [SMOTE, BorderlineSMOTE, ADASYN]

lr = LogisticRegression()

for over in over_sampling:

    for under in under_sampling:

        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        for strategy in strategy_:
            
            if under == OneSidedSelection:
                under_ = under(random_state=42)
            else:
                under_ = under()

            over_ = over(sampling_strategy=strategy, random_state=42)

            ss = StandardScaler()
            x_train_ss = ss.fit_transform(x_train)
            x_test_ss = ss.fit_transform(x_test)

            model = make_pipeline(over_, under_, lr)   
            model.fit(x_train_ss, y_train)
            pred = model.predict(x_test_ss)

            acc = accuracy_score(y_test, pred)
            recall = recall_score(y_test, pred)
            precision = precision_score(y_test, pred)
            f1 = f1_score(y_test, pred)

            df.loc[over.__name__+under.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("Logistic Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Logistic Model's score by sampling


,f1_score,recall,precision,accuracy
SMOTETomekLinks(0.3),0.942857,0.951736,0.934142,0.903258
SMOTEOneSidedSelection(0.3),0.942549,0.951148,0.934104,0.902764
ADASYNTomekLinks(0.3),0.941348,0.949382,0.933449,0.900790
ADASYNOneSidedSelection(0.3),0.941039,0.948793,0.933411,0.900296
BorderlineSMOTETomekLinks(0.3),0.939554,0.942319,0.936805,0.898322
BorderlineSMOTEOneSidedSelection(0.3),0.938931,0.941142,0.936731,0.897335
SMOTENeighbourhoodCleaningRule(0.3),0.934446,0.922896,0.946288,0.891412
BorderlineSMOTEOneSidedSelection(0.4),0.934211,0.919364,0.949544,0.891412
BorderlineSMOTETomekLinks(0.4),0.934211,0.919364,0.949544,0.891412
SMOTEOneSidedSelection(0.4),0.934086,0.925839,0.942481,0.890424


## Under, Over, Combine Sampling과 Pipeline을 통한 Combine Sampling 전체 결과

In [8]:
sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    if i in [OneSidedSelection, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]:
        sampling = i(random_state=42)
    else:
        sampling = i()
        
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    lr = LogisticRegression(random_state=42)
    lr.fit(x_train_ss, y_train)
    pred = lr.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[i.__name__] = [f1, recall, precision, acc]

print("Logistic Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

Logistic Model's score by sampling


,f1_score,recall,precision,accuracy
SMOTETomekLinks(0.3),0.942857,0.951736,0.934142,0.903258
SMOTEOneSidedSelection(0.3),0.942549,0.951148,0.934104,0.902764
OneSidedSelection,0.942324,0.966451,0.919373,0.900790
TomekLinks,0.942324,0.966451,0.919373,0.900790
NeighbourhoodCleaningRule,0.941725,0.951148,0.932487,0.901283
ADASYNTomekLinks(0.3),0.941348,0.949382,0.933449,0.900790
ADASYNOneSidedSelection(0.3),0.941039,0.948793,0.933411,0.900296
BorderlineSMOTETomekLinks(0.3),0.939554,0.942319,0.936805,0.898322
BorderlineSMOTEOneSidedSelection(0.3),0.938931,0.941142,0.936731,0.897335
EditedNearestNeighbours,0.937793,0.940553,0.935050,0.895360


## NO Sampling

In [9]:
columns = ['f1_score', 'recall', 'precision', 'accuracy']
NO_Sampling = pd.DataFrame(columns=columns)

x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.fit_transform(x_test)


lr = LogisticRegression(random_state=42)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

acc = accuracy_score(y_test, pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
f1 = f1_score(y_test, pred)

NO_Sampling.loc['No Sampling'] = [f1, recall, precision, acc]

print("Logistic Model's score by sampling")
NO_Sampling.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
NO_Sampling.head(10)

Logistic Model's score by sampling


,f1_score,recall,precision,accuracy
No Sampling,0.922947,0.962331,0.886659,0.865252
